<a href="https://colab.research.google.com/github/Pinlinzz/Analisis-Dataset-Employee-Salary-Analysis-/blob/main/Analisis_Dataset_Employee_Salary_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INISIASI DAN IMPORTING DATASET


In [ ]:
# Load the dataset into an R data frame
salary_df <- read.csv('/content/employee-salary-analysis-dataset/Salary_Dataset.csv')

# Display the first 5 rows of the data frame
head(salary_df)

,Employee_ID,Name,Age,Gender,Country,City,Education,Job_Title,Department,Experience_Years,Salary_USD,Bonus_USD,Work_Hours_Per_Week,Remote_Work,Performance_Score,Joining_Year,Contract_Type
,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>
1,1,Employee_1,58,Female,India,NY,Bachelor,Manager,IT,0,130735,3650,41,Yes,1,2015,Part-Time
2,2,Employee_2,48,Female,UK,NY,High School,Designer,HR,4,54363,14600,52,No,2,2022,Part-Time
3,3,Employee_3,34,Male,USA,London,High School,Manager,IT,1,76694,15317,51,Yes,8,2010,Part-Time
4,4,Employee_4,27,Male,UK,Karachi,Bachelor,Developer,Marketing,19,31582,19768,51,Yes,5,2004,Contract
5,5,Employee_5,40,Male,Canada,London,Master,Manager,IT,2,109910,18193,59,Yes,7,2008,Full-Time
6,6,Employee_6,58,Female,India,London,PhD,Designer,Marketing,18,96844,16563,34,Yes,1,2002,Full-Time
